# 2020.03.13 DACON 온도 추정 경진대회


## Baseline code 작성

## 1. 라이브러리 및 데이터

In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
import sys
sys.path.append("/content/gdrive/My Drive/Colab Notebooks")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
# tensorflow 2.0 업데이트, gpu활용을 위해 tensorflow-gpu 설치
!pip install tensorflow
!pip install tensorflow-gpu

     |████████████████████████████████| 421.8MB 39kB/s 
     |████████████████████████████████| 3.9MB 64.4MB/s 
     |████████████████████████████████| 450kB 55.2MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.1.1 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.1.0 which is incompatible.
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1


In [0]:
import os
os.chdir('./gdrive/My Drive/Colab Notebooks/2020_1_data/02.dacon_temp')

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import tensorflow as tf
import random
import os

# load the data 
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")
total_data = pd.concat([train,test], sort=False)

print(train.shape)
print(test.shape)

(4752, 60)
(11520, 41)


## 2. Data Cleansing & Pre-processing

In [14]:
# X00 ~ X39 추출(기상청 데이터)
X_train = train.loc[:, 'X00':'X39']
test = test.loc[:, 'X00':'X39']
print(X_train.shape)

# standardization
MEAN = X_train.mean()
STD = X_train.std()

# 표준편차가 0일 경우 대비해 추가
X_train = (X_train - MEAN) / (STD + 1e-07)
print(X_train.shape)

(4752, 40)
(4752, 40)


In [0]:
# RNN 모델에 입력할 수 있는 시계열 형태로 데이터 변환
def convert_to_timeseries(df, interval):
  sequence_list = []
  target_list = []

  for i in tqdm(range(df.shape[0] - interval)):
    sequence_list.append(np.array(df.iloc[i:i+interval,:-1]))
    target_list.append(df.iloc[i+interval, -1])

  sequence = np.array(sequence_list)
  target = np.array(target_list)

  return sequence, target

토론 글에 기반하여 Y15, Y16 지점의 데이터가 Y18과 상관관계가 높음. 그래서 두 지점의 데이터를 학습하기로 결정.

In [0]:
y_columns = ['Y15', 'Y16']

In [17]:
# t시점 이전 120분의 데이터로 t시점의 온도를 추정할 수 있는 학습데이터 생성
sequence = np.empty((0, 12, 40))
target = np.empty((0,))

for column in y_columns:
  concat = pd.concat([X_train, train[column]], axis = 1)
  _sequence, _target = convert_to_timeseries(concat.head(144*30), interval=12)
  sequence = np.vstack((sequence, _sequence))
  target = np.hstack((target, _target))

100%|██████████| 4308/4308 [00:00<00:00, 4497.90it/s]


In [19]:
# dummy feature 생성
X_train['dummy'] = 0

# standardization 실시
test = (test-MEAN) / (STD+1e-07)

test['dummy'] = 0

# train, test 기간을 합쳐서 120분 간격으로 학습데이터 재구축
X_test, _ = convert_to_timeseries(pd.concat([X_train, test], axis= 0), interval=12)

print()
print(X_train.shape)
print(test.shape)
print(X_test.shape)

100%|██████████| 16260/16260 [00:10<00:00, 1562.22it/s]



(4752, 41)
(11520, 41)
(16260, 12, 40)


In [20]:
#print(X_test.shape)

# test set 기간인 후반부 80일에 맞게 자르기
X_test = X_test[-11520:, : , :]

# dummy feature 제거
X_train.drop('dummy', axis=1, inplace=True)
test.drop('dummy', axis=1, inplace=True)

print()
print(X_train.shape)
print(test.shape)
print(X_test.shape)


(4752, 40)
(11520, 40)
(11520, 12, 40)


## 3. Feature Enginnering & Initial Modeling
### 변수 선택 및 모델 구축

In [0]:
# 간단한 lstm 모델 구축
simple_lstm_model = tf.keras.models.Sequential([tf.keras.layers.LSTM(128, input_shape=sequence.shape[-2:]),
                                                tf.keras.layers.Dense(256, activation='linear'),
                                                tf.keras.layers.Dense(128, activation='linear'),
                                                tf.keras.layers.Dense(1)
])

simple_lstm_model.compile(optimizer='adam', loss='mse')

## 4. Model Tuning & Evaluation
### 모델 학습 및 검증

In [0]:
# loss가 2미만으로 떨어지면 학습 종료시키는 기능
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    if(logs.get('loss')<2):
      print("\n Loss is under 2, cancelling training")
      self.model.stop_training = True

In [0]:
callbacks = myCallback()

In [24]:
simple_lstm_model.fit(sequence, target,
                      epochs=60,
                      batch_size=128,
                      verbose=2,
                      shuffle=False,
                      callbacks = [callbacks]
)

Train on 8616 samples
Epoch 1/60
8616/8616 - 3s - loss: 157.8664
Epoch 2/60
8616/8616 - 2s - loss: 15.5737
Epoch 3/60
8616/8616 - 2s - loss: 8.0518
Epoch 4/60
8616/8616 - 2s - loss: 6.2045
Epoch 5/60
8616/8616 - 2s - loss: 5.6361
Epoch 6/60
8616/8616 - 2s - loss: 5.5144
Epoch 7/60
8616/8616 - 2s - loss: 5.4940
Epoch 8/60
8616/8616 - 2s - loss: 5.6167
Epoch 9/60
8616/8616 - 2s - loss: 5.4980
Epoch 10/60
8616/8616 - 2s - loss: 5.2690
Epoch 11/60
8616/8616 - 2s - loss: 5.1743
Epoch 12/60
8616/8616 - 2s - loss: 5.3149
Epoch 13/60
8616/8616 - 2s - loss: 4.9242
Epoch 14/60
8616/8616 - 2s - loss: 5.1666
Epoch 15/60
8616/8616 - 2s - loss: 4.4794
Epoch 16/60
8616/8616 - 2s - loss: 5.3715
Epoch 17/60
8616/8616 - 2s - loss: 6.3745
Epoch 18/60
8616/8616 - 2s - loss: 7.9080
Epoch 19/60
8616/8616 - 2s - loss: 6.9109
Epoch 20/60
8616/8616 - 2s - loss: 6.0063
Epoch 21/60
8616/8616 - 2s - loss: 5.2657
Epoch 22/60
8616/8616 - 2s - loss: 5.7582
Epoch 23/60
8616/8616 - 2s - loss: 5.2002
Epoch 24/60
8616/8

In [25]:
# LSTM layer 고정
simple_lstm_model.layers[0].trainable = False

# fine tuning 사용할 학습데이터 생성(Y18)
finetune_X, finetune_Y = convert_to_timeseries(pd.concat([X_train.tail(432), train['Y18'].tail(432)], axis=1), interval=12)

100%|██████████| 420/420 [00:00<00:00, 2443.34it/s]


In [26]:
# LSTM layer는 고정 시켜두고, DNN 레이어에 대해서 fine tuning 진행 (Transfer Learning)
finetune_history = simple_lstm_model.fit(
    finetune_X, finetune_Y,
    epochs = 20,
    batch_size=64,
    shuffle=False,
    verbose=2
)

Train on 420 samples
Epoch 1/20
420/420 - 0s - loss: 7.9613
Epoch 2/20
420/420 - 0s - loss: 3.9280
Epoch 3/20
420/420 - 0s - loss: 3.5249
Epoch 4/20
420/420 - 0s - loss: 2.6944
Epoch 5/20
420/420 - 0s - loss: 1.9501
Epoch 6/20
420/420 - 0s - loss: 1.5984
Epoch 7/20
420/420 - 0s - loss: 1.3726
Epoch 8/20
420/420 - 0s - loss: 1.1489
Epoch 9/20
420/420 - 0s - loss: 0.9919
Epoch 10/20
420/420 - 0s - loss: 0.8479
Epoch 11/20
420/420 - 0s - loss: 0.7948
Epoch 12/20
420/420 - 0s - loss: 0.6888
Epoch 13/20
420/420 - 0s - loss: 0.6751
Epoch 14/20
420/420 - 0s - loss: 0.5829
Epoch 15/20
420/420 - 0s - loss: 0.5566
Epoch 16/20
420/420 - 0s - loss: 0.4913
Epoch 17/20
420/420 - 0s - loss: 0.4656
Epoch 18/20
420/420 - 0s - loss: 0.4164
Epoch 19/20
420/420 - 0s - loss: 0.3964
Epoch 20/20
420/420 - 0s - loss: 0.3625


In [27]:
print(X_test.shape)
print(X_train.shape)

(11520, 12, 40)
(4752, 40)


In [0]:
# 예측하기
finetune_pred = simple_lstm_model.predict(X_test)

# 제출 파일 만들기
submit = pd.DataFrame({'id': range(144*33, 144*113),
                       'Y18': finetune_pred.reshape(1,-1)[0]})

submit.to_csv('baseline_res.csv', index=False)